User story 4.<br><br>
Om te bepalen welk model we gaan gebruiken moeten we weten wat het verschil is in die modellen en wat het beste past bij het probleem en wat in de beste score resulteerd. 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime as date

from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Variable prep

In [3]:
usefullcols = ['stm_mon_nr','stm_sap_meld_ddt','stm_status_melding_sap','stm_km_van_mld','stm_km_tot_mld','stm_km_van_gst',
'stm_km_tot_gst','stm_fh_tijd','stm_sap_melddatum','stm_aanngeb_tijd','stm_aanntpl_tijd','stm_arbeid',
'stm_progfh_in_tijd','stm_progfh_in_invoer_tijd','stm_progfh_in_duur','stm_sap_storeindtijd','stm_progfh_gw_tijd',
'stm_reactie_duur','stm_progfh_gw_duur','stm_progfh_gw_teller','stm_afspr_aanvangdd','stm_afspr_aanvangtijd',
'stm_fh_duur','stm_evb','stm_sap_meldtijd','stm_sap_meldtekst_lang','stm_prioriteit','stm_oh_pg_gst',
'stm_sap_meldtekst','stm_techn_gst','stm_contractgeb_gst','stm_tao_indicator','stm_geo_mld','stm_functiepl_mld',
'stm_geo_mld_uit_functiepl','stm_aanngeb_ddt','stm_aanngeb_dd','stm_oorz_code','stm_oorz_groep','stm_oorz_tkst',
'stm_fh_dd','stm_fh_status','stm_geo_gst','stm_functiepl_gst','stm_geo_gst_uit_functiepl','stm_fh_ddt',
'stm_aanntpl_dd','stm_techn_mld','stm_sap_storeinddatum','stm_equipm_nr_mld','stm_equipm_soort_mld',
'stm_equipm_omschr_mld','stm_sap_storeind_ddt','stm_contractgeb_mld','stm_equipm_nr_gst','stm_equipm_soort_gst',
'stm_equipm_omschr_gst','stm_progfh_in_invoer_dat','stm_progfh_in_datum','stm_oorz_tekst_kort','stm_dstrglp_naar',
'stm_tao_indicator_vorige','stm_vl_post','stm_dstrglp_van','stm_pplg_van','stm_tao_soort_mutatie',
'stm_progfh_gw_lwd_tijd','stm_pplg_naar','stm_progfh_gw_lwd_datum']

In [4]:
df = pd.read_csv('sap_storing_data_hu_project.csv',usecols=usefullcols, low_memory=True)

/Users/jet/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,24,27,49,51,55,56,73,75,78,80,82,86,87,91,95,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
features = ['stm_prioriteit','stm_oorz_code','stm_oorz_groep','stm_equipm_nr_*','stm_equipm_soort_*']
target = 'stm_fh_duur'

In [6]:
print(df['stm_fh_duur'].dtypes)
display(df[['stm_fh_duur']])
display(df[['stm_fh_duur']].describe())

float64


,stm_fh_duur
0,NaN
1,0.0
2,51.0
3,40.0
4,6.0
...,...
898521,83.0
898522,83.0
898523,83.0
898524,19.0


,stm_fh_duur
count,898488.000000
mean,923.148771
std,6404.546787
min,0.000000
25%,11.000000
50%,84.000000
75%,256.000000
max,528297.000000


In [7]:
now = date.datetime.today()
zeroTimeDiff = now - now

df['stm_sap_meld_ddt'] = pd.to_datetime(df['stm_sap_meld_ddt'], format='%d/%m/%Y %H:%M:%S')
df['stm_sap_storeind_ddt'] = pd.to_datetime(df['stm_sap_storeind_ddt'], format='%d/%m/%Y %H:%M:%S')
# df['stm_fh_tijd'] = pd.to_timedelta(df['stm_fh_tijd'], unit='s')


# Meldingen waar de oplossing in de toekomst is zijn onzin.
df[df['stm_sap_storeind_ddt'] >= now][['stm_sap_storeind_ddt','stm_sap_meld_ddt']]
df.drop(df[df['stm_sap_storeind_ddt'] >= now][['stm_sap_storeind_ddt','stm_sap_meld_ddt']].index,inplace=True)

# Een datetime diff om extra waardes te kunnen filteren.
# df['diff_meld_strend'] = df['stm_sap_storeind_ddt']-df['stm_sap_meld_ddt']

# Rijen waar het probleem eerder is opgelost dan de meldingen zijn onbetrouwbaar.
# df.drop(df[df['diff_meld_strend'] < zeroTimeDiff].index, inplace = True)

# Alles wat langer dan een dag duurt is niet urgent genoeg en dus niet nuttig.
# df.drop(df[df['diff_meld_strend'] > pd.to_timedelta(6, unit='h')].index, inplace = True)
# df.drop(df[df['stm_fh_tijd'] > pd.to_timedelta(6, unit='h')].index, inplace = True)
df.drop(df[df['stm_fh_duur'] > 360].index, inplace = True)

# Waar geen duur is hebben we niks aan dus die moeten weg.
df.dropna(subset=['stm_fh_duur'],inplace=True)

Merge similar columns together.

In [8]:
dubbleCol = ['stm_equipm_omschr_*', 'stm_equipm_soort_*', 'stm_equipm_nr_*', 
             'stm_geo_*_uit_functiepl', 'stm_functiepl_*', 'stm_geo_*',
            'stm_km_van_*','stm_km_tot_*',
            'stm_contractgeb_*','stm_techn_*']
original = 'mld'
optional = ['gst'] # Order of least to most important
for colPH in dubbleCol:
    colOg = colPH.replace('*',original)
    
    for option in optional:
        colOp = colPH.replace('*',option)
        df[colPH] = np.where(df[colOp].isna(), df[colOg], df[colOp])
        df.drop(columns=[colOp],inplace=True)
    df.drop(columns=[colOg],inplace=True)

Het vullen van interresante colommen.
Correcte type waardes geven aan de colommen.

In [9]:
df['stm_prioriteit'] = df['stm_prioriteit'].fillna(1) # Laagste prio
df['stm_oorz_code'] = df['stm_oorz_code'].fillna(299) # 'Niet gemeld' code
df['stm_oorz_groep'] = df['stm_oorz_groep'].fillna('x')
df['stm_fh_status'] = df['stm_fh_status'].fillna(1) # Median of values
df['stm_aanngeb_tijd'] = df['stm_aanngeb_tijd'].fillna(df['stm_sap_meldtijd']) # Aannemer beltijd gelijk met melding zetten.
df['stm_progfh_in_invoer_tijd'] = df['stm_progfh_in_invoer_tijd'].fillna(df['stm_aanngeb_tijd']) # Tijd van prognose op beltijd zetten.
df['stm_equipm_nr_*'] = df['stm_equipm_nr_*'].fillna(0)
df['stm_equipm_soort_*'] = df['stm_equipm_soort_*'].fillna('x')
df['stm_contractgeb_*'] = df['stm_contractgeb_*'].fillna(0)
df['stm_progfh_gw_teller'] = df['stm_progfh_gw_teller'].fillna(0)

df['stm_prioriteit'] = df['stm_prioriteit'].astype(int)
df['stm_oorz_code'] = df['stm_oorz_code'].astype(int)
df['stm_fh_status'] = df['stm_fh_status'].astype(int)
df['stm_fh_duur'] = df['stm_fh_duur'].astype(int)
df['stm_progfh_gw_teller'] = df['stm_progfh_gw_teller'].astype(int)
df['stm_equipm_nr_*'] = df['stm_equipm_nr_*'].astype(int)
df['stm_contractgeb_*'] = df['stm_contractgeb_*'].astype(int)

Baseline model 

In [10]:
amntInTime = df[df['stm_fh_duur'] < 137].count()[0]
total = df.shape[0]
print(f'{amntInTime / total * 100}% van de tijd heeft het baseline model het goed.')

79.23289331896551% van de tijd heeft het baseline model het goed.


In [11]:
le_ozgr = LabelEncoder()
le_ozgr.fit(df['stm_oorz_groep'].unique())
df['stm_oorz_groep'] = le_ozgr.transform(df['stm_oorz_groep'])
# df['stm_oorz_groep'].unique()

In [12]:
le_equipsrt = LabelEncoder()
le_equipsrt.fit(df['stm_equipm_soort_*'].unique())
df['stm_equipm_soort_*'] = le_equipsrt.transform(df['stm_equipm_soort_*'])
# df['stm_equipm_soort_*'].unique()

In [13]:
features

['stm_prioriteit',
 'stm_oorz_code',
 'stm_oorz_groep',
 'stm_equipm_nr_*',
 'stm_equipm_soort_*']

In [ ]:
dOC = pd.get_dummies(df['stm_oorz_code'])
dOG = pd.get_dummies(df['stm_oorz_groep'])
dP = pd.get_dummies(df['stm_prioriteit'])
equipnr = pd.get_dummies(df['stm_equipm_nr_*'])
eqs = pd.get_dummies(df['stm_equipm_soort_*'])
dummies = [dOC, dOG, dP, equipnr, eqs]

In [ ]:
X = dummies
#X = df[features]
Y = df[target]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=10)

In [ ]:
linearModel = linear_model.LinearRegression().fit(x_train,y_train)
linYPrediction = linearModel.predict(x_test)
print(f'Score: {linearModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,linYPrediction))}')

In [ ]:
display(x_test[:5])
display(y_test[:5])
linYPrediction[:5]

In [ ]:
ridgeModel = linear_model.Ridge(alpha=1).fit(x_train,y_train)
ridgeYPrediction = ridgeModel.predict(x_test)
print(f'Score: {ridgeModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,ridgeYPrediction))}')

In [ ]:
knearestModel = KNeighborsRegressor(n_neighbors=2).fit(x_train,y_train)
knearestYPrediction = knearestModel.predict(x_test)
print(f'Score: {knearestModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,knearestYPrediction))}')

In [ ]:
# Nc = range(1, 10)
# kmeans = [KNeighborsRegressor(n_neighbors=i) for i in Nc]
# kmeans
# score = [kmeans[i].fit(x_train,y_train).score(x_test,y_test) for i in range(len(kmeans))]
# score
# plt.plot(Nc,score)
# plt.xlabel('Number of Clusters')
# plt.ylabel('Score')
# plt.title('Elbow Curve')
# plt.show()

## Non-linear regression trees

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
dtree = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)

dtree.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import r2_score

# Code lines 1 to 3
pred_train_tree= dtree.predict(x_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_tree)))
print(r2_score(y_train, pred_train_tree))

# Code lines 4 to 6
pred_test_tree= dtree.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_tree))) 
print(r2_score(y_test, pred_test_tree))

In [ ]:
dtree1 = DecisionTreeRegressor(max_depth=2)
dtree2 = DecisionTreeRegressor(max_depth=37)
dtree1.fit(x_train, y_train)
dtree2.fit(x_train, y_train)

# Code Lines 5 to 6: Predict on training data
tr1 = dtree1.predict(x_train)
tr2 = dtree2.predict(x_train) 

#Code Lines 7 to 8: Predict on testing data
y1 = dtree1.predict(x_test)
y2 = dtree2.predict(x_test) 

In [ ]:
print(np.sqrt(mean_squared_error(y_train,tr1))) 
print(r2_score(y_train, tr1))

In [ ]:
print(np.sqrt(mean_squared_error(y_train,tr2))) 
print(r2_score(y_train, tr2))


In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)

In [ ]:
model_rf.fit(x_train, y_train) 

In [ ]:
pred_train_rf= model_rf.predict(x_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))
